In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <tuple>
#include <torch/torch.h>
namespace nn = torch::nn;

# RNN

Applies a multi-layer Elman RNN with \tanhtanh or \text{ReLU}ReLU non-linearity to an input sequence

https://github.com/pytorch/pytorch/blob/master/torch/csrc/api/include/torch/nn/modules/rnn.h

https://github.com/pytorch/pytorch/blob/master/torch/csrc/api/include/torch/nn/options/rnn.h

In [3]:
int sample_size = 2;
int sequence_length = 3;
int number_feature = 4;

torch::Tensor x = torch::arange(sample_size*sequence_length*number_feature).reshape({sample_size,sequence_length, number_feature}) * 1.0;

In [4]:
std::cout << x << std::endl;

(1,.,.) = 
   0   1   2   3
   4   5   6   7
   8   9  10  11

(2,.,.) = 
  12  13  14  15
  16  17  18  19
  20  21  22  23
[ CPUFloatType{2,3,4} ]


In [5]:
int output_size = 5;
int number_layers = 1;
auto options = nn::RNNOptions(/*input_size*/number_feature, /*hidden_size4*/output_size).num_layers(number_layers).batch_first(true);

In [6]:
nn::RNN rnn_function(options);

In [7]:
torch::Tensor h0 = torch::zeros({number_layers, sample_size, output_size});

std::tuple<torch::Tensor, torch::Tensor> output = rnn_function(x, h0);

In [8]:
torch::Tensor rnn_output, hn;
std::tie(rnn_output, hn) = output; //unpacking tuple into variables

@0x7fff31a3b5c8

In [9]:
std::cout << rnn_output << std::endl;

(1,.,.) = 
 -0.8987 -0.8704 -0.8842 -0.6157  0.8737
 -0.9991 -1.0000 -0.5193 -0.7914  0.9889
 -1.0000 -1.0000  0.1340 -0.9031  0.9990

(2,.,.) = 
 -1.0000 -1.0000  0.8248 -0.9864  0.9999
 -1.0000 -1.0000  0.8984 -0.9795  1.0000
 -1.0000 -1.0000  0.9803 -0.9935  1.0000
[ CPUFloatType{2,3,5} ]


In [10]:
std::cout << hn << std::endl;

(1,.,.) = 
 -1.0000 -1.0000  0.1340 -0.9031  0.9990
 -1.0000 -1.0000  0.9803 -0.9935  1.0000
[ CPUFloatType{1,2,5} ]


In [11]:
// hn = output[:,-1,:]
using torch::indexing::Slice;

std::cout << rnn_output.index({Slice(),-1, Slice()}) << std::endl;

-1.0000 -1.0000  0.1340 -0.9031  0.9990
-1.0000 -1.0000  0.9803 -0.9935  1.0000
[ CPUFloatType{2,5} ]
